In [1]:
import pyvista as pv
from dataclasses import dataclass
import pandas as pd
import numpy as np

In [2]:
@dataclass
class planet():
    name:str
    initial_position:np.array
    initial_velocity:np.array
    mass:int
    

In [3]:
terra = planet(
    name="Terra",
    initial_position=np.array([0,0,0]),
    initial_velocity=np.array([0,0,0]),
    mass=6e24,
)

lua = planet(
    name="Lua",
    initial_position=np.array([380e6,0,0]),
    initial_velocity=np.array([0,0,0]),
    mass=7e22,
)

planets_list = [terra,lua]

In [22]:
tf = 2*24 #horas
time_step = 60 #secs
time_vector = np.linspace(0,60*60*tf,endpoint=False,num=60*60*tf//time_step)


def compute_aceleration_vector(body_position,body_mass,list_other_bodies_positions,list_other_bodies_masses):
    G = 6.67*10**(-11) # Nm2/kg2

    force_vector = np.zeros(3)

    for other_body_position,other_body_mass in zip(list_other_bodies_positions,list_other_bodies_masses):
        vector_r = other_body_position-body_position
        force_vector += G*body_mass*other_body_mass/np.abs(np.sum(vector_r**3))*vector_r

    aceleration = force_vector/body_mass
    
    return aceleration

dict_planets_history = {}
for planet in planets_list:
    df = pd.DataFrame(columns=["time","position","velocity"])
    df = df.set_index("time")
    df.loc[time_vector[0]] = {
        "position":planet.initial_position,
        "velocity":planet.initial_velocity,
    }
    dict_planets_history.update({planet.name:df})

dict_planets_history
for time in time_vector:
    for planet in planets_list:
        aceleration = compute_aceleration_vector(
            body_position=dict_planets_history[planet.name].loc[time,"position"],
            body_mass=planet.mass,
            list_other_bodies_positions = [
                other_planet_df.loc[time,"position"] 
                for other_planet_name,other_planet_df 
                in dict_planets_history.items() 
                if other_planet_name != planet.name
            ],
            list_other_bodies_masses = [
               other_planet.mass for other_planet in planets_list if other_planet != planet
            ],
        )

        velocity = dict_planets_history[planet.name].loc[time,"velocity"]
        position = dict_planets_history[planet.name].loc[time,"position"]

        new_velocity = velocity + time_step * aceleration
        new_position = position + time_step * (new_velocity+velocity)/2

        dict_planets_history[planet.name].loc[time+time_step] = {
            "position":new_position,
            "velocity":new_velocity,
        }


KeyError: 60.0

In [21]:
dict_planets_history["Terra"]

,position,velocity
time,,
0.0,"[0, 0, 0]","[0, 0, 0]"
60.0,"[0.05820083102493076, 0.0, 0.0]","[0.0019400277008310254, 0.0, 0.0]"
120.0,"[0.232803325645673, 0.0, 0.0]","[0.0038800554531937155, 0.0, 0.0]"
180.0,"[0.5238074900460269, 0.0, 0.0]","[0.005820083360151415, 0.0, 0.0]"
240.0,"[0.9312133365935946, 0.0, 0.0]","[0.007760111524767509, 0.0, 0.0]"
...,...,...
172560.0,"[500602.6437640361, 0.0, 0.0]","[6.045299269121741, 0.0, 0.0]"
172620.0,"[500965.4359011374, 0.0, 0.0]","[6.047771967588957, 0.0, 0.0]"
172680.0,"[501328.3764140153, 0.0, 0.0]","[6.050245128339873, 0.0, 0.0]"


In [14]:
raio_terra = 6370 #km
raio_lua = 1735 #km

mesh_terra = pv.Sphere(radius=raio_terra,)
mesh_lua = pv.Sphere(radius=raio_lua,center=lua.initial_position)

cena = pv.Plotter()

cena.add_mesh(mesh_terra)
cena.add_mesh(mesh_lua)

cena.show()

Widget(value='<iframe src="http://localhost:63634/index.html?ui=P_0x1bc86fb74a0_3&reconnect=auto" class="pyvis…